In [1]:
import mammoth
from IPython.core.display import display, HTML
import subprocess
import os
import glob
import string
import stat
import random
from bs4 import BeautifulSoup
import re
from difflib import SequenceMatcher
import shutil 
import filecmp
import difflib
import pickle

In [2]:
def stripInfo(soup):
    regExpList = [r'Coffman', r'149 Atlantic',r'Swampscott',r'\$\d*\.\d\d']
    for thisRegExp in regExpList:
        for elem in soup(text=re.compile(thisRegExp)):
            elem.parent.decompose()

In [3]:
# CONVERT TO HTML, DELETE SENSITIVE INFORMATION (ADDRESS, ORDER INFO, ETC....), SAVE HTML FILE

allDocFilesGlob = "/Users/mettinger/Desktop/rockArtData/**/*.docx"
allFiles = glob.glob(allDocFilesGlob, recursive=True)
convertErrors = []
regExErrors = []
writeErrors = []
print(len(allFiles))

for i,filename in enumerate(allFiles):
    if i % 200 == 0:
        print(i)
    with open(filename, "rb") as docx_file:
        try:
            result = mammoth.convert_to_html(docx_file)
            html = result.value
            soup = BeautifulSoup(html, 'html.parser')
            try:
                stripInfo(soup)
                html = str(soup)
                try:
                    thisFileName = filename[0:-5] + ".html"
                    with open(thisFileName, "w") as thisHtmlFile:
                        thisHtmlFile.write(str(html))
                except:
                    writeErrors.append(filename)
            except:
                regExErrors.append(filename)
        except:
            convertErrors.append(filename)
    
len(writeErrors), len(regExErrors), len(convertErrors)

2432
0
200
400
600
800
1000
1200
1400
1600
1800
2000
2200
2400


(0, 6, 35)

In [7]:
# MEASURE SIMILARITY OF ALL FILENAMES TO DETECT DUPLICATES AND NEAR DUPLICATES

allFilesGlob = "/Users/mettinger/Desktop/rockArtData/**/*.html"
allFiles = glob.glob(allFilesGlob, recursive=True)
print(len(allFiles))

simNameList = []

for i in range(len(allFiles)):
    if i % 200 == 0:
        print(i)
    firstPath = allFiles[i]
    firstName = firstPath.split('/')[-1].split('.')[0]
    for j in range(i + 1, len(allFiles)):
        secondPath = allFiles[j]
        secondName = secondPath.split('/')[-1].split('.')[0]
        score = SequenceMatcher(None, firstName, secondName).ratio()
        simNameList.append((firstPath, secondPath, score))
        
simNameList = sorted(simNameList, key = lambda x: x[2], reverse = True)

print("identical filenames: " + str(len([i for i in simNameList if i[2] == 1])))

2391
0
200
400
600
800
1000
1200
1400
1600
1800
2000
2200
identical filenames: 5


In [8]:
# MEASURE TO DETECT DUPLICATES AND NEAR DUPLICATES

allFilesGlob = "/Users/mettinger/Desktop/rockArtData/**/*.html"
allFiles = glob.glob(allFilesGlob, recursive=True)
print(len(allFiles))

simHTMLList = []

for i,firstPath in enumerate(allFiles):
    if i % 200 == 0:
        print(i)
    for j in range(i+1, len(allFiles)):
        secondPath = allFiles[j]
        score = SequenceMatcher(None, firstPath, secondPath).ratio()
        simHTMLList.append((firstPath, secondPath, score))
        
simHTMLList = sorted(simHTMLList, key = lambda x: x[2], reverse = True)

2391
0
200
400
600
800
1000
1200
1400
1600
1800
2000
2200


In [9]:
# MAKE LIST OF SETS OF "EQUIVALENT" FILES

def equivExtend(equivRelation, equivList, cutoffScore):
    for file1, file2, score in equivList:
        if score >=  cutoffScore:
            blockList = []
            equivRelation.append(set([file1, file2]))
            for blockIndex,thisBlock in enumerate(equivRelation):
                if (file1 in thisBlock) or (file2 in thisBlock):
                    blockList.append(blockIndex)
            equivRelation[blockList[0]] = equivRelation[blockList[0]].union(*[equivRelation[i] for i in blockList])
            
            
            for i in blockList[:0:-1]:
                del equivRelation[i]
                
            ''' 
            blockList.reverse()
            for i in blockList[0:-1]:
                del equivRelation[i]
            '''
    return equivRelation


a = equivExtend([], simNameList, .95)
a = equivExtend(a, simHTMLList, .95)

print("Number of sets of equivalent files: " + str(len(a)))
print()

index = 15
for thisPath in a[index]:
    print(thisPath)

Number of sets of equivalent files: 286

/Users/mettinger/Desktop/rockArtData/2018_AS_Cathy_In_Process/Case 5_AS/OLD OR NOT IN CASE 5/A357-Asia-Thailand-Ban Chiang-Bangle-Shell-3600 BCE.html
/Users/mettinger/Desktop/rockArtData/2018_AS_Cathy_In_Process/Case 5_AS/OLD OR NOT IN CASE 5/A357-Asia-Thailand-Ban Chiang-Bangles-Shell-3600 BCE.html


In [13]:
# WRITE SIMILARITY FILE

simList = a

with open('/Users/mettinger/Desktop/similarList.txt', 'w') as f:
    for thisSet in simList:
        for item in thisSet:
            item = '/Users/tip/'+ '/'.join(item.split('/')[3:])
            f.write("%s\n" % item)
        f.write("\n\n")